## Demand Survey and Surplus ##

The idea for this mini-lab is to derive a Demand curve from student input from the first day of class, and to use that to motivate the concept of surplus.


In [ ]:
from datascience import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches
import sympy
solve = lambda x,y: sympy.solve(x-y)[0] if len(sympy.solve(x-y))==1 else "Not Single Solution"
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
plt.style.use("seaborn-muted")

!pip install oauth2client
import pandas as pd
import gspread
import os
import json
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np
from datascience import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


Lets start off with the demand from a student survey:
 - we had 4 "goods" and a range of prices available for each good
 - students made bids on their willingness to pay for each of the 4 goods
 - the dataset for Fall 2021 has something like 83 observations
 - this dataset has been exported and we have to read it in here:

In [ ]:
# This cell loads in the data from the google form and saves it to a table called demand_table - don't worry about the code!
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_dict(json.loads(os.environ['GOOGLE_SHEETS_READONLY_KEY']), scope)
gc = gspread.authorize(creds)

url = 'https://docs.google.com/spreadsheets/d/16iLCfbeLQRG_GiEM5Tc0t4VYxsl9-pTIm-Kx4LaZxGI/edit?usp=sharing'

sheet = gc.open_by_url(url)
ws = sheet.worksheet('Form Responses 1')
df = pd.DataFrame(ws.get_all_records())

DemandTable = Table.from_df(df)
DemandTable = DemandTable.drop('Timestamp')
DemandTable


In [ ]:
for i in DemandTable.labels:
    DemandTable.hist(i);

Lets start by looking at just a burrito

And let's try to figure how how many people would buy a gourmet burrito at a given price? Let's assume that a person would be willing to buy the good at a price less than their bid price.  

In [ ]:
BurritosTable=DemandTable.select('Burrito')
BurritosTable

In [ ]:
# Count how many people are in each answer pool
BurritosTable.group("Burrito")

Now the number of each isnt the right quantity - we want to know the number of people who would pay at any given price or any lower price
 - people will pay "up to their bid price"
 - we consider the bid the maximum they are willing to pay, but will pay anything less
 - the bid was for a single unit, so for now we will disregard individuals buying multiple units
 
 We are going to do this by 
  1) flipping the order of the count column
  2) doing a cumulative function of all prices below
  3) flipping it back

In [ ]:
#Cumulative Sum of how many people are willing to pay at a agiven price
Qdemand = np.flip(np.cumsum(np.flip(BurritosTable.group("Burrito").column("count"))))

In [ ]:
# Combine the two into a single table
DemandBurr= Table().with_columns([
    'priceBurr',[2.5, 5, 7.5, 10.00, 12.5, 15,17.5],
    'Qdemand',Qdemand
])
DemandBurr

## Lets looks at this Table and think about Consumer Surplus
Lets start from by flipping the order so that it is descending

And then let's suppose the price is $10 and lets measure how many people that were willing to pay more are getting a "surplus" by getting to by at this price that is lower than their "Willingness to Pay" / " Bid Price"


In [ ]:
DemandBurr.sort("priceBurr", descending = True)

It looks like 
- 28 people would have been willing to pay up to $ 12.5

- 6 people would have been willing to pay up to $ 15.0

- 1 person would have been willing to pay up to $ 17.5

- 0 people would have been willing to pay up to $ 20.0

Lets add up these values


In [ ]:
CS_counting = 28*2.5+6*5+1*7.5+0*10
print('The consumer surplus from counting consumers is ', CS_counting )

In [ ]:
DemandBurr.plot("Qdemand", "priceBurr")
plt.xlabel('Quantity')
plt.ylabel('Price')
plt.title('Demand for Gourmet Burrito');

In [ ]:
DemandBurr.scatter("Qdemand", "priceBurr")
plt.xlabel('Quantity')
plt.ylabel('Price')
plt.title('Demand for Gourmet Burrito');

Now that we have a demand curve - Let's create for a model that makes a linear approximation like we did in lecture 2

In [ ]:
#What are the slope and intercept of the fit line
std_units = lambda a: (a - np.mean(a)) / np.std(a)
corr = lambda x, y: np.mean(std_units(x) * std_units(y))
slope = lambda x, y: corr(x, y) * np.std(y) / np.std(x)
intercept = lambda x, y: np.mean(y) - slope(x, y) * np.mean(x)

In [ ]:
burr_slope = slope(DemandBurr["Qdemand"], DemandBurr["priceBurr"])
burr_slope

In [ ]:
burr_intercept = intercept(DemandBurr["Qdemand"], DemandBurr["priceBurr"])
burr_intercept

In [ ]:
DemandBurr.plot("Qdemand", "priceBurr")
plt.xlabel('Quantity')
plt.ylabel('Price')
plt.title('Demand for Burrito');
plt.plot(np.arange(0,81,0.01), burr_slope * np.arange(0,81,0.01) + burr_intercept);

In [ ]:
#How many people does the Model think would buy at $10
solve = lambda x,y: sympy.solve(x-y)[0] if len(sympy.solve(x-y))==1 else "Not Single Solution"
Q = sympy.Symbol("Q")
demand = burr_slope * Q + burr_intercept
supply = 10
Q_Star=solve(demand,supply)
#Q_star = solve(demand, supply) # our version of solve is simplified for single solution systems
Q_Star

In [ ]:
DemandBurr.plot("Qdemand", "priceBurr")
plt.xlabel('Quantity')
plt.ylabel('Price')
plt.title('Demand for Burrito')
plt.plot(np.arange(0,82,0.01), burr_slope * np.arange(0,82,0.01) + burr_intercept)
plt.hlines(10,0,45, color ='r')
#plt.hlines(y = 1, xmin = 0, xmax = 82, color ='r')

#model_q= burr_slope *1 + burr_intercept)
triangle1 = patches.Polygon([[0,10],[Q_Star,10],[0,17.15]],True,color="green")
currentAxis = plt.gca()
currentAxis.add_patch(triangle1)

print("Consumer surplus is equal to green triangle: "+str(0.5*(17.15-10)*45))


In [ ]:
DemandBurr.sort("priceBurr", descending = True)

##  Let's try again for Greek Theater Tickets 

In [ ]:
GreekTixTable=DemandTable.select('GreekTix')
GreekTixTable


In [ ]:
Qdemand = np.flip(np.cumsum(np.flip(GreekTixTable.group("GreekTix").column("count"))))


In [ ]:
DemandGreekTix= Table().with_columns([
    'priceTix',[25, 50, 75, 100, 125, 150,175, 200],
    'Qdemand',Qdemand
])
DemandGreekTix

In [ ]:
DemandGreekTix.plot("Qdemand", "priceTix")
plt.xlabel('Quantity')
plt.ylabel('Price')
plt.title('Demand for Greek Theater Tickets');

## Section 2  - fun with widgets

In [ ]:
import sympy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
p = sympy.Symbol("p")
def Equilibrium(demandParam, supplyParam, priceStart):
    demandEquation = demandParam - p
    # change the slope
    supplyEquation = p * (supplyParam/10)
    priceEnd = sympy.solve(demandEquation)[0]
    prices = []
    demandQ = []
    supplyQ = []
    for price in range(priceStart,priceEnd+1):
        prices += [price]
        demandQ += [demandEquation.subs(p,price)]
        supplyQ += [supplyEquation.subs(p,price)]
    
    equilibriumP = sympy.solve(demandEquation-supplyEquation)[0]
    equilibriumQ = demandEquation.subs(p,equilibriumP)
    
    
    
    triangle1 = patches.Polygon([[equilibriumQ,equilibriumP],[0,equilibriumP],[0,priceEnd]],True,color="green")
    triangle2 = patches.Polygon([[equilibriumQ,equilibriumP],[0,equilibriumP],[0,0]],True,color="red")
    currentAxis = plt.gca()
    currentAxis.add_patch(triangle1)
    currentAxis.add_patch(triangle2)
    
    plt.plot(demandQ,prices)
    plt.plot(supplyQ,prices)
    plt.legend(["Demand","Supply"])
    plt.plot(equilibriumQ,equilibriumP, 'ro')
    plt.xlabel("Supply and Demand Quantity")
    plt.ylabel("Price")
    plt.ylim(0, 15)
    plt.xlim(0, 10)
    plt.show()
    print("The equilibrium price is "+str(round(equilibriumP,2))+" and equilibrium quantity is "+str(round(equilibriumQ,2))+".")
    print("The consumer surplus at this equilibrium "+str((priceEnd-equilibriumP)*(equilibriumQ)*.5))
    print("The producer surplus at this equilibrium "+str((equilibriumP)*(equilibriumQ)*.5))
# you can change the range here
slider1 = widgets.IntSlider(min=5, max=15,step=1,value=10)
slider2 = widgets.IntSlider(min=1, max=20,step=1,value=10)
slider3 = widgets.IntSlider(min=-5, max=5,step=1,value=0)
display(widgets.interactive(Equilibrium, demandParam=slider1, supplyParam=slider2, priceStart=slider3))

In [ ]:
def eqSolve(eq1,eq2,tax):
    demandP = sympy.solve(eq1-q,p)[0]
    supplyP = sympy.solve(eq2-q,p)[0]
    demandP = demandP-cTax
    supplyP = supplyP+pTax

    demandQ = sympy.solve(demandP-p,q)[0]
    supplyQ = sympy.solve(supplyP-p,q)[0]
    
    return sympy.solve((demandP-supplyP, demandQ-supplyQ,tax-cTax-pTax), q,p,cTax,pTax)[q]

In [ ]:
import sympy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
p = sympy.Symbol("p")
q = sympy.Symbol("q")
cTax = sympy.Symbol("cTax")
pTax = sympy.Symbol("pTax")

def EquilibriumTax(demandParam,supplyParam,priceStart,priceEnd,tax):
    demandEquation = demandParam - p
    supplyEquation = p * (supplyParam/10)
    prices = []
    demand = []
    supply = []
    for price in range(priceStart,priceEnd+1):
        prices += [price]
        demand += [demandEquation.subs(p,price)]
        supply += [supplyEquation.subs(p,price)]
        
    
    
    nonTaxPrice = sympy.solve(demandEquation-supplyEquation)[0]
    nonTaxQ = demandEquation.subs(p,nonTaxPrice)

    
    equilibriumQ = eqSolve(demandEquation,supplyEquation,tax)
    equilibriumP1 = sympy.solve(demandEquation-equilibriumQ)[0]
    equilibriumP2 = sympy.solve(supplyEquation-equilibriumQ)[0]
    
    triangle1 = patches.Polygon([[nonTaxQ,nonTaxPrice],[equilibriumQ,nonTaxPrice],[equilibriumQ,equilibriumP1]],True,color="green")
    triangle2 = patches.Polygon([[nonTaxQ,nonTaxPrice],[equilibriumQ,nonTaxPrice],[equilibriumQ,equilibriumP2]],True)
    currentAxis = plt.gca()
    currentAxis.add_patch(triangle1)
    currentAxis.add_patch(triangle2)
    
    
    rect1 = patches.Rectangle((0,nonTaxPrice),equilibriumQ,equilibriumP1-nonTaxPrice,linewidth=1,facecolor="red")
    rect2 = patches.Rectangle((0,nonTaxPrice),equilibriumQ,equilibriumP2-nonTaxPrice,linewidth=1,facecolor="yellow")
    currentAxis.add_patch(rect1)
    currentAxis.add_patch(rect2)
    
    plt.plot(demand,prices)
    plt.plot(supply,prices)
    
    
    plt.legend([rect1,rect2,triangle1,triangle2], ["Consumer Tax","Producer Tax","Consumer Deadweight Loss","Producer Deadweight Loss"])
    plt.plot(equilibriumQ,equilibriumP1, 'ro')
    plt.plot(equilibriumQ,equilibriumP2, 'ro')
    plt.xlabel("Supply and Demand Quantity")
    plt.ylabel("Price")
    plt.ylim(0, 15)
    plt.xlim(0, 10)
    plt.show()
    print("Without Tax - the equilibrium price is "+str(round(nonTaxPrice,2))+" and equilibrium quantity is "+str(round(nonTaxQ,2)))
    print("With Tax - Price paid by consumers is "+str(equilibriumP1)+" Price received by suppliers is "+str(round(equilibriumP2,2))+" and equilibrium quantity is "+str(equilibriumQ)+".")
    print("Taxes raised from consumers equals "+str(round(equilibriumQ*(equilibriumP1-nonTaxPrice),2)))
    print("Taxes raised from producers equals "+str(round(equilibriumQ*(nonTaxPrice-equilibriumP2),2)))
    print("Total taxes raised equals "+str(equilibriumQ*tax))

slider1 = widgets.IntSlider(min=5, max=15,step=1,value=10)
slider2 = widgets.IntSlider(min=1, max=20,step=1,value=10)
slider3 = widgets.IntSlider(min=-5, max=5,step=1,value=0)
slider4 = widgets.IntSlider(min=5, max=20,step=1,value=10)
slider5 = widgets.IntSlider(min=0, max=8,step=1,value=4)
display(widgets.interactive(EquilibriumTax, demandParam=slider1, supplyParam=slider2, priceStart=slider3, priceEnd=slider4, tax=slider5))